In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [60]:
from tf_lab.in_out.basics import Data_Splitter
from tf_lab.point_clouds.in_out import PointCloudDataSet, load_point_clouds_from_filenames
from tf_lab.data_sets.shape_net import pc_loader as snc_loader

def load_multiple_version_of_pcs(version, syn_id, n_classes, n_pc_points=2048, random_seed=42, verbose=False):
    top_data_dir = '/orions4-zfs/projects/optas/DATA/'

    if n_classes != 1:
        raise ValueError()

    if version == 'uniform_all':
        versions = ['centered', 'centered_2nd_version', 'centered_3rd_version']

    elif version == 'uniform_one':
        versions = ['centered_2nd_version']

    elif version == 'fps':
        versions = ['fps_sampled_in_u_sphere']

    elif version == 'all':
        versions = ['centered', 'centered_2nd_version', 'centered_3rd_version', 'fps_sampled_in_u_sphere']
    else:
        raise ValueError()

    splits = {'train': None, 'val': None, 'test': None}

    for s in splits.keys():
        if verbose:
            print 'Loading %s data.' % (s,)
        s_file = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits/' + syn_id + '/85_5_10/', s + '.txt')
        if verbose:
            print s_file
        for _, v in enumerate(versions):
            top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/', v, str(n_pc_points))
            splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply', random_seed=random_seed)
            pcs_in_split = splitter.load_splits(s_file)
            pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(pcs_in_split, n_threads=20, loader=snc_loader, verbose=True)
            if splits[s] is None:
                splits[s] = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)
            else:
                splits[s].merge(PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids))
    return splits

In [58]:
syn_id = snc_category_to_synth_id()[class_name]

In [59]:
pc1 = load_multiple_version_of_pcs('uniform_one', syn_id, 1)

679 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


In [61]:
pc2 = load_multiple_version_of_pcs('uniform_one', syn_id, 1)

679 pclouds were loaded. They belong in 1 shape-classes.
5761 pclouds were loaded. They belong in 1 shape-classes.
338 pclouds were loaded. They belong in 1 shape-classes.


In [29]:
from geo_tool import Point_Cloud

In [62]:
# Point_Cloud(pc1['val'].point_clouds[0]).plot()

In [46]:
from tf_lab.evaluate.generative_pc_nets import minimum_mathing_distance

In [66]:
from numpy.random import normal

In [72]:
a = pc1['val'].point_clouds
b = pc1['val'].point_clouds + normal(scale=0.01, size=(pc1['val'].point_clouds.shape))

minimum_mathing_distance(a, b, 400, use_EMD=True)


(35.087219,
 [34.723442,
  35.695347,
  36.675575,
  36.299862,
  36.576176,
  36.610443,
  33.687553,
  35.823833,
  32.715092,
  33.860409,
  34.391418,
  34.020638,
  36.538696,
  36.603127,
  34.145569,
  32.687431,
  34.919136,
  34.271439,
  35.385338,
  33.543182,
  35.644321,
  36.644218,
  36.589172,
  35.568981,
  34.550556,
  35.196049,
  36.569439,
  34.897373,
  36.615227,
  35.231445,
  36.014973,
  36.706623,
  35.100304,
  36.343403,
  32.959991,
  35.297695,
  33.059456,
  34.912079,
  32.972694,
  36.582432,
  34.757996,
  34.292076,
  36.431187,
  33.572906,
  34.122711,
  36.752098,
  35.418022,
  34.304108,
  35.334755,
  34.281158,
  35.77523,
  34.768677,
  34.693523,
  35.746964,
  34.716461,
  31.94095,
  36.670368,
  35.225296,
  34.62088,
  35.625114,
  35.344288,
  32.093803,
  36.663601,
  35.015839,
  34.942677,
  35.610249,
  34.487663,
  36.315826,
  34.554848,
  34.900078,
  33.007477,
  35.185349,
  35.406219,
  36.227665,
  35.791111,
  32.413921,
  3

In [44]:
# c = 0
# for pc in pc1['val'].point_clouds:
#     p = Point_Cloud(pc)
#     if not p.is_in_unit_sphere():
#         c += 1
# print c

0


In [ ]:
# DO SOME tests of 'scaling' between pc1 and pc2. Then run MMD-CD between 'val-set'.

In [9]:
from tf_lab.data_sets.shape_net import snc_category_to_synth_id

In [10]:
class_name = 'chair'

do_jsd = False
do_mmd = True

save_res = False

top_out_dir = top_evaluation_dir
experiment_name = ''

sample_mmd = False
n_samples = 2
ref_pop = 2000
sample_pop = 2000
mmd_loss = 'emd'
special_tag = 'decoder'
skip = ['val', 'test']

In [6]:
if do_jsd:
    boost_samples = 1
else:
    boost_samples = 1
assert(np.logical_xor(do_jsd, do_mmd))

In [7]:
# top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]

In [7]:
if save_res:
    exp_name = osp.basename(top_sample_dir)
    if do_jsd:
        f_jsd = open(osp.join(top_out_dir, class_name, 'jsd_' + special_tag + exp_name + '.txt'), 'w')
            
    if do_mmd:
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + exp_name + '.txt'), 'w')
else:
    f_jsd = sys.stdout
    f_mmd = sys.stdout

In [9]:
sample_files = ['/orions4-zfs/projects/optas/DATA/OUT/iclr/evaluations/decoded_gt_data_aligned_to_gt/chair/mlp_with_split_1pc_usampled_bnorm_on_encoder_only/emd_bneck_128/train.npz']

In [10]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [ ]:
for sample_file in sample_files:
    evaluator.prepare_sample_data(sample_file, boost_samples)
    sample_name = osp.basename(sample_file)[:-len('.npz')]
    print sample_name
    if do_jsd:
        f_jsd.write(sample_name + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        f_mmd.write(sample_name + '\n')
        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                              ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                              f_out=f_mmd, skip=skip)
        f_mmd.flush()
    print

train
train


In [12]:
if save_res:
    if do_jsd:
        f_jsd.close()
    if do_mmd:
        f_mmd.close()